In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

# julia

> An implementation of the Julia set.

In [ ]:
#| default_exp fractal.julia

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#from abc import abstractmethod
import numpy as np
#from fractalart.core import Image
from numba import njit, prange
#from numba import njit, prange, jit
import math
#import matplotlib.pyplot as plt
from fractalart.fractal.abstract_fractal import Fractal

In [ ]:
#| export
@njit(parallel=True, fastmath=True)
def _compute_fractal(x_min: float, x_max: float, y_min: float, y_max: float, resolution: tuple[int, int],
                     max_iter: int, fractal_fn, order: int = 1, smooth: bool = True) -> np.ndarray:
    
    width, height = resolution
    result = np.zeros((height, width), dtype=np.float64)
    r2_cut = max(abs(x_max), abs(x_min)) * max(abs(x_max), abs(x_min)) + max(abs(y_max), abs(y_min)) * max(abs(y_max), abs(y_min))

    dx = (x_max - x_min) / (width - 1)
    dy = (y_max - y_min) / (height - 1)
    inv_log2 = 1.0 / math.log(2.0)

    for j in prange(height):
        zy = y_min + j * dy
        for i in range(width):
            zx = x_min + i * dx
            zr = 0.0
            zi = 0.0
            cr = zx
            ci = zy
            iteration = 0

            while zr * zr + zi * zi <= r2_cut and iteration < max_iter:
                zr, zi = fractal_fn(zr, zi, cr, ci, order)
                iteration += 1

            if smooth and iteration < max_iter:
                result[j, i] = smooth_coloring(zr, zi, iteration)
            else:
                result[j, i] = iteration

    return result

In [ ]:
#| export
_inv_log2 = 1.0 / math.log(2.0)

@njit
def smooth_coloring(zr, zi, iteration):
    mag_sq = zr * zr + zi * zi

    # Ensure mag_sq > 1 to avoid log(≤0)
    if mag_sq > 1e-8:
        log_zn = 0.5 * math.log(mag_sq)
        if log_zn > 1e-8:
            nu = math.log(log_zn * _inv_log2) * _inv_log2
            return iteration + 1 - nu

    return float(iteration)
    
@njit
def julia_step(zr, zi, cr, ci):
    return mandelbrot_step(zr, zi, cr, ci)

In [ ]:
##| export
#@njit(parallel=True, fastmath=True)
#def _compute_julia_set(xmin, xmax, ymin, ymax, width, height, cr, ci, max_iter):
#    dx = (xmax - xmin) / (width - 1)
#    dy = (ymax - ymin) / (height - 1)
#    result = np.zeros((height, width), dtype=np.float32)
#
#    for j in prange(height):
#        y0 = ymin + j * dy
#        for i in range(width):
#            x0 = xmin + i * dx
#            x = x0
#            y = y0
#            iteration = 0
#            while x*x + y*y <= 6.25 and iteration < max_iter:
#                x_temp = x*x - y*y + cr
#                y = 2.0 * x * y + ci
#                x = x_temp
#                iteration += 1
#
#            if iteration < max_iter:
#                # Smooth coloring
#                modulus_sq = x*x + y*y
#                log_zn = np.log(modulus_sq) / 2
#                nu = np.log(log_zn / np.log(2)) / np.log(2)
#                result[j, i] = iteration + 1 - nu
#            else:
#                result[j, i] = max_iter
#
#    return result
#
#class Julia(Fractal):
#    def __init__(
#        self,
#        cr: float = -0.7,
#        ci: float = 0.27015,
#        width: int = 800,
#        height: int = 800,
#        x_min: float = -1.5,   # override Fractal default
#        x_max: float =  1.5,   # override Fractal default
#        y_min: float = -1.5,   # keep same (or change)
#        y_max: float =  1.5,   # keep same (or change)
#        max_iter: int = 1000,  # keep same (or change)
#    ):
#        # 1) initialize the Fractal portion:
#        super().__init__(width, height, x_min, x_max, y_min, y_max, max_iter)
#
#        # 2) store Julia‐specific constants
#        self._cr = cr
#        self._ci = ci
#        
#    def compute(self) -> np.ndarray:
#        w, h = self.resolution
#        # pass resolution-consistent dims
#        return _compute_julia_set(self._x_min, self._x_max, self._y_min, self._y_max, w, h, self._cr, self._ci, self._max_iter)

In [ ]:
#j = Julia(cr = -0.7, ci = 0.27015)
#j.resolution = 1200, 1200
#j.max_iter = 3000
#j.render()
#j.equalize_histogram()
#j.plot()

In [ ]:
##| export
#@njit(parallel=True, fastmath=True)
#def _compute_julia_cross_trap(xmin, xmax, ymin, ymax, width, height, cr, ci, max_iter):
#    dx = (xmax - xmin) / (width - 1)
#    dy = (ymax - ymin) / (height - 1)
#    cross_trap = np.zeros((height, width), dtype=np.float32)
#
#    for j in prange(height):
#        y0 = ymin + j * dy
#        for i in range(width):
#            x0 = xmin + i * dx
#            x = x0
#            y = y0
#            iteration = 0
#            min_cross = 1e10
#            
#            while iteration < max_iter:
#                x_temp = x*x - y*y + cr
#                y = 2.0 * x * y + ci
#                x = x_temp
#                iteration += 1
#
#                # Cross trap: distance to real or imaginary axis
#                # TOTO : only take the min after a specific number of iterations , e. g. iteration > 3:
#                if iteration > -1:
#                    cross_dist = min(abs(x), abs(y))
#                    if (cross_dist < min_cross):
#                        min_cross = cross_dist
#
#            cross_trap[j, i] = min_cross
#
#    return cross_trap
#
#class JuliaCrossTrap(Fractal):
#    def __init__(
#        self,
#        cr: float = -0.7,
#        ci: float = 0.27015,
#        width: int = 800,
#        height: int = 800,
#        x_min: float = -1.5,   # override Fractal default
#        x_max: float =  1.5,   # override Fractal default
#        y_min: float = -1.5,   # keep same (or change)
#        y_max: float =  1.5,   # keep same (or change)
#        max_iter: int = 1000,  # keep same (or change)
#    ):
#        # 1) initialize the Fractal portion:
#        super().__init__(width, height, x_min, x_max, y_min, y_max, max_iter)
#
#        # 2) store Julia‐specific constants
#        self._cr = cr
#        self._ci = ci
#        
#    def compute(self) -> np.ndarray:
#        w, h = self.resolution
#        # pass resolution-consistent dims
#        return _compute_julia_cross_trap(self._x_min, self._x_max, self._y_min, self._y_max, w, h, self._cr, self._ci, self._max_iter)

In [ ]:
#j = JuliaCrossTrap(cr = 0.355, ci = 0.355)
#j.resolution = 1200, 1200
#j.max_iter = 3000
#j.render()
#j.equalize_histogram()
#j.plot()

In [ ]:
#j = Julia(cr = -0.4, ci = 0.6)
#j.resolution = 1200, 1200
#j.max_iter = 3000
#j.render()
#j.equalize_histogram()
#j.plot()

In [ ]:
#j = Julia(cr = -0.8, ci = 0.156)
#j.resolution = 1200, 1200
#j.max_iter = 3000
#j.render()
#j.equalize_histogram()
#j.plot()

In [ ]:
#m = Mandelbrot()
#m.resolution = 1200, 1200
#m.max_iter = 3000
#m.set_zoom(1, (0.0, 0.0))
#m.render()
#m.equalize_histogram()
#m.plot()

In [ ]:
#m = MandelbrotCrossTrap()
#m.resolution = 1200, 1200
#m.max_iter = 3000
#m.set_zoom(2, (0.0, 0.0))
#m.render()
#m.equalize_histogram()
#m.plot()

In [ ]:
#m = MandelbrotCrossTrap()
#m.resolution = 1200, 1200
#m.max_iter = 3000
#
##m.set_zoom(5, (-0.170337,-1.06506))
##m.set_zoom(25, (-0.170337,-1.06506))
##m.set_zoom(125, (-0.170337,-1.06506))
##m.set_zoom(625, (-0.170337,-1.06506))
##m.set_zoom(3125, (-0.170337,-1.06506))
##m.set_zoom(15625, (-0.170337,-1.06506))
##m.set_zoom(78125, (-0.170337,-1.06506))
#
##m.set_zoom(5, (0.42884,-0.231345))
##m.set_zoom(25, (0.42884,-0.231345))
#m.set_zoom(125, (0.42884,-0.231345))
##m.set_zoom(625, (0.42884,-0.231345))
##m.set_zoom(3125, (0.42884,-0.231345))
##m.set_zoom(15625, (0.42884,-0.231345))
##m.set_zoom(78125, (0.42884,-0.231345))
#
##m.set_zoom(5, (-1.62917,-0.0203968))
##m.set_zoom(25, (-1.62917,-0.0203968))
##m.set_zoom(125, (-1.62917,-0.0203968))
##m.set_zoom(625, (-1.62917,-0.0203968))
##m.set_zoom(3125, (-1.62917,-0.0203968))
##m.set_zoom(15625, (-1.62917,-0.0203968))
##m.set_zoom(78125, (-1.62917,-0.0203968))
#
##m.set_zoom(5, (-0.761574,-0.0847596))
##m.set_zoom(25, (-0.761574,-0.0847596))
##m.set_zoom(125, (-0.761574,-0.0847596))
##m.set_zoom(625, (-0.761574,-0.0847596))
##m.set_zoom(3125, (-0.761574,-0.0847596))
##m.set_zoom(15625, (-0.761574,-0.0847596))
##m.set_zoom(78125, (-0.761574,-0.0847596))
#
#m.render()
#m.equalize_histogram()
#m.plot()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()